In [ ]:
# https://www.kaggle.com/c/santa-2019-revenge-of-the-accountants/overview
# https://www.kaggle.com/golubev/mip-optimization-preference-cost-santa2019revenge
# https://www.kaggle.com/vipito/fork-of-santa-ip

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/santa-2019-revenge-of-the-accountants/family_data.csv
/kaggle/input/santa-2019-revenge-of-the-accountants/sample_submission.csv


In [2]:
!pip install -U pulp

     |████████████████████████████████| 39.2MB 355kB/s 


In [3]:
fpath = '/kaggle/input/santa-2019-revenge-of-the-accountants/family_data.csv'
data = pd.read_csv(fpath, index_col='family_id')

fpath = '/kaggle/input/santa-2019-revenge-of-the-accountants/sample_submission.csv'
submission = pd.read_csv(fpath, index_col='family_id')

In [4]:
data.head()

,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
family_id,,,,,,,,,,,
0,68,4,26,3,45,80,61,89,6,31,6
1,66,17,82,4,29,52,46,77,75,60,2
2,89,23,5,27,1,24,53,11,9,17,2
3,66,80,35,59,4,96,87,94,1,33,3
4,29,52,24,33,27,2,45,20,12,21,3


In [5]:
submission.head()

,assigned_day
family_id,
0,100
1,99
2,98
3,97
4,96


In [6]:
import numpy as np
import pandas as pd
from numba import njit
from itertools import product


def get_penalty(n, choice):
    penalty = None
    if choice == 0:
        penalty = 0
    elif choice == 1:
        penalty = 50
    elif choice == 2:
        penalty = 50 + 9 * n
    elif choice == 3:
        penalty = 100 + 9 * n
    elif choice == 4:
        penalty = 200 + 9 * n
    elif choice == 5:
        penalty = 200 + 18 * n
    elif choice == 6:
        penalty = 300 + 18 * n
    elif choice == 7:
        penalty = 300 + 36 * n
    elif choice == 8:
        penalty = 400 + 36 * n
    elif choice == 9:
        penalty = 500 + 36 * n + 199 * n
    else:
        penalty = 500 + 36 * n + 398 * n
    return penalty

def preference_cost_matrix(data, days=100):
    preference_matrix = np.zeros((n_families, days), dtype=np.int64)
    for i in range(n_families):
        desired = data.values[i, :-1]
        preference_matrix[i, :] = get_penalty(family_sizes[i], 10)
        for j, day in enumerate(desired):
            preference_matrix[i, day-1] = get_penalty(family_sizes[i], j)

    return preference_matrix

def accounting_penalty_matrix():
    accounting_matrix = np.zeros((500, 500, 6), dtype=np.float64)
    for n in range(accounting_matrix.shape[0]):
        for n_p1 in range(accounting_matrix.shape[1]):
            for j in range(1, 6):
                diff = abs(n - n_p1)
                value = max(0, (n - 125) / 400 * (n**(0.5 + diff / 50.0)) / j**2) 
                accounting_matrix[n, n_p1, j] = value
                
    return accounting_matrix

In [7]:
days = 100
n_families = data.shape[0]
family_sizes = data.n_people.values

MAX_OCCUPANCY = 300
MIN_OCCUPANCY = 125

desired = data.values[:, :-1] - 1
preference_matrix = preference_cost_matrix(data)
accounting_matrix = accounting_penalty_matrix()

In [8]:
from pulp import *

thrs = 8

indexs = []
candidates = [[] for _ in range(days)]

for i in range(data.shape[0]):
    for j in desired[i, :]:
        candidates[j].append(i)
        indexs.append((i, j))
        
prob = LpProblem('Santa', LpMinimize)

x = LpVariable.dicts('x', indexs, lowBound=0, cat='Continuous')

daily_occupancy = [lpSum([x[i, j] * family_sizes[i] for i in candidates[j]]) for j in range(days)]
family_presence = [lpSum([x[i, j] for j in desired[i, :]]) for i in range(n_families)]

# Objective
prob += lpSum([preference_matrix[i, j] * x[i, j] for i in range(n_families) for j in desired[i, :]])

# Constraints
for j in range(days-1):
    prob += lpSum(daily_occupancy[j] - daily_occupancy[j+1]) <= thrs
    prob += lpSum(daily_occupancy[j+1] - daily_occupancy[j]) <= thrs
    
for i in range(n_families):
    prob += family_presence[i] == 1

for j in range(days):
    prob += daily_occupancy[j] >= MIN_OCCUPANCY
    prob += daily_occupancy[j] <= MAX_OCCUPANCY

In [9]:
prob.solve()
print('Status', LpStatus[prob.status])
print('Z = {}'.format(value(prob.objective)))

Status Optimal
Z = 117354.58602520998


In [10]:
tmp = [(i, j, x[i, j].value()) for i in range(n_families) for j in desired[i, :] if x[i, j].value() > 0]
df = pd.DataFrame(tmp, columns=['family_id', 'day', 'n'])
print(len(df))

THRS = 0.999
assigned_df   = df[df.n > THRS].copy()
unassigned_df = df[(df.n <= THRS) & (df.n > 1-THRS)]
unassigned = unassigned_df.family_id.unique()
print('{} unassigned families'.format(len(unassigned)))

6086
84 unassigned families


In [11]:
assigned_df['family_size'] = family_sizes[assigned_df.family_id]
occupancy = assigned_df.groupby('day').family_size.sum().values
min_occupancy = np.array([max(0, MIN_OCCUPANCY-o) for o in occupancy])
max_occupancy = np.array([MAX_OCCUPANCY - o for o in occupancy])
print(min_occupancy)
print(max_occupancy)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 15   4   3   2   3   1   4   5  16   6   6   4  14   2   2   6   0   8
   4  12   0   6  10   2   3   7   8  13  13   8   3   8   4  12  22  22
  10   4   3  12  16  23  20  18   1   9   8  14  19  36  17  18  12  26
  27  35  42  50  68  63  82  79  88  99  89  84  74  85  89  95 105 108
 100  95 103 113 119 124 119 110 100 100 103 115 121 115 106  99 107 112
 120 136 121 114 123 129 138 145 152 160]


In [12]:
n_families = len(unassigned)
indexs = []
candidates = [[] for _ in range(days)]

for i in unassigned:
    for j in desired[i, :]:
        candidates[j].append(i)
        indexs.append((i, j))
        
prob = LpProblem('Santa', LpMinimize)

x = LpVariable.dicts('x', indexs, lowBound=0, cat='Binary')

daily_occupancy = [lpSum([x[i, j] * family_sizes[i] for i in candidates[j]]) for j in range(days)]
family_presence = [lpSum([x[i, j] for j in desired[i, :]]) for i in unassigned]

# Objective
prob += lpSum([preference_matrix[i, j] * x[i, j] for i in unassigned for j in desired[i, :]])

# Constraints    
for i in range(n_families):
    prob += family_presence[i] == 1

for j in range(days):
    prob += daily_occupancy[j] >= min_occupancy[j]
    prob += daily_occupancy[j] <= max_occupancy[j]

In [13]:
prob.solve()
print('Status', LpStatus[prob.status])
print('Z = {}'.format(value(prob.objective)))

Status Optimal
Z = 2475.0


In [14]:
tmp = [(i, j) for i in unassigned for j in desired[i, :] if x[i, j].value() > 0]
df = pd.DataFrame(tmp, columns=['family_id', 'day'])
print(len(df))

84


In [15]:
len(assigned_df)

5916

In [16]:
df = pd.concat((assigned_df[['family_id', 'day']], df)).sort_values('family_id')
print(len(df))

6000


In [17]:
from numba import njit


@njit(fastmath=True)
def preference_cost(prediction, days=100):
    daily_occupancy = np.zeros(days, dtype=np.int64)
    penalty = 0
    for (i, p) in enumerate(prediction):
        n = family_sizes[i]
        penalty += preference_matrix[i, p]
        daily_occupancy[p] += n
        
    return penalty, daily_occupancy

@njit(fastmath=True)
def accounting_penalty(daily_occupancy, days=100):
    do = np.zeros(days+5, dtype=np.int64)
    do[:days] = daily_occupancy
    do[days:] = do[days-1]
    accounting_cost = 0
    n_out_of_range = 0
    for day in range(days):
        for j in range(1, 6):
            n_pj = do[day + j]
            n    = do[day]
            n_out_of_range += (n > MAX_OCCUPANCY) or (n < MIN_OCCUPANCY)
            accounting_cost += accounting_matrix[n, n_pj, j]
            
    return accounting_cost, n_out_of_range

@njit(fastmath=True)
def cost_function(prediction):
    penalty, daily_occupancy = preference_cost(prediction)
    accounting_cost, n_out_of_range = accounting_penalty(daily_occupancy)
    return penalty + accounting_cost + n_out_of_range * 100000000

In [18]:
pc, occ = preference_cost(df.day.values)
ac, oor = accounting_penalty(occ)
print('{}, {:.2f}, ({}, {})'.format(pc, ac, occ.min(), occ.max()))

115642, 3718.74, (140, 300)


In [19]:
def findBetterDay4Family(pred):
    fobs = np.argsort(family_sizes)
    score = cost_function(pred)
    original_score = np.inf
    
    while original_score > score:
        original_score = score
        for family_id in fobs:
            for pick in range(10):
                day = desired[family_id, pick]
                oldvalue = pred[family_id]
                pred[family_id] = day
                new_score = cost_function(pred)
                if new_score < score:
                    score = new_score
                else:
                    pred[family_id] = oldvalue

        print(score, end='\r')
    print(score)

new = df.day.values.copy()
findBetterDay4Family(new) 

116457.38398817397


In [20]:
def stochastic_product_search(top_k, fam_size, original, 
                              verbose=1000, verbose2=50000,
                              n_iter=3000, random_state=42):
    """
    original (np.array): The original day assignments.
    
    At every iterations, randomly sample fam_size families. Then, given their top_k
    choices, compute the Cartesian product of the families' choices, and compute the
    score for each of those top_k^fam_size products.
    """
    
    best = original.copy()
    best_score = cost_function(best)
    
    np.random.seed(random_state)

    for i in range(n_iter):
        fam_indices = np.random.choice(range(desired.shape[0]), size=fam_size)
        changes = np.array(list(product(*desired[fam_indices, :top_k].tolist())))

        for change in changes:
            new = best.copy()
            new[fam_indices] = change

            new_score = cost_function(new)

            if new_score < best_score:
                best_score = new_score
                best = new
                
        if verbose and i % verbose == 0:
            print(f"Iteration #{i}: Best score is {best_score:.2f}      ", end='\r')
            
        if verbose2 and i % verbose2 == 0:
            print(f"Iteration #{i}: Best score is {best_score:.2f}      ")
    
    print(f"Final best score is {best_score:.2f}")
    return best

In [21]:
MAX_OCCUPANCY = 300
MIN_OCCUPANCY = 125

findBetterDay4Family(new) 

116457.38398817397


In [22]:
final = stochastic_product_search(
    top_k=3,
    fam_size=8, 
    original=new, 
    n_iter=15000,
    verbose=1000,
    verbose2=50000,
    random_state=42
)

Iteration #0: Best score is 116457.38      
Final best score is 116197.69


In [23]:
def seed_finding(seed, prediction_input):
    prediction = prediction_input.copy()
    np.random.seed(seed)
    best_score = cost_function(prediction)
    original_score = best_score
    best_pred = prediction.copy()
    print("SEED: {}   ORIGINAL SCORE: {}".format(seed, original_score))
    for t in range(100):
        for i in range(5000):
            for j in range(10):
                di = prediction[i]
                prediction[i] = desired[i, j]
                cur_score = cost_function(prediction)

                KT = 1
                if t < 5:
                    KT = 1.5
                    
                elif t < 10:
                    KT = 4.5
                else:
                    if cur_score > best_score + 100:
                        KT = 3
                        
                    elif cur_score > best_score + 50 :
                        KT = 2.75
                        
                    elif cur_score > best_score + 20:
                        KT = 2.5
                        
                    elif cur_score > best_score + 10:
                        KT = 2
                        
                    elif cur_score > best_score:
                        KT = 1.5
                    else:
                        KT = 1

                prob = np.exp(-(cur_score - best_score) / KT)
                if np.random.rand() < prob:
                    best_score = cur_score
                else:
                    prediction[i] = di
                    
        if best_score < original_score:
            print("NEW BEST SCORE on seed {}: {}".format(seed, best_score))
            original_score = best_score
            best_pred = prediction.copy()

    return best_pred

In [24]:
final = seed_finding(42, final)

SEED: 42   ORIGINAL SCORE: 116197.68647634466
NEW BEST SCORE on seed 42: 116191.26586966851
NEW BEST SCORE on seed 42: 116185.9101916494
NEW BEST SCORE on seed 42: 116182.39782913805
NEW BEST SCORE on seed 42: 116157.15896635197
NEW BEST SCORE on seed 42: 116152.2458372156
NEW BEST SCORE on seed 42: 116117.24519900486
NEW BEST SCORE on seed 42: 116116.49116974801
NEW BEST SCORE on seed 42: 116101.8212690173
NEW BEST SCORE on seed 42: 116100.71676887688
NEW BEST SCORE on seed 42: 116099.64282624779
NEW BEST SCORE on seed 42: 116097.83061715661
NEW BEST SCORE on seed 42: 116097.10522520663
NEW BEST SCORE on seed 42: 116077.5385013484
NEW BEST SCORE on seed 42: 116060.12635929494
NEW BEST SCORE on seed 42: 116058.36900130585
NEW BEST SCORE on seed 42: 116057.75247448357
NEW BEST SCORE on seed 42: 116053.65218973954
NEW BEST SCORE on seed 42: 116052.00650466491
NEW BEST SCORE on seed 42: 116050.71610870633
NEW BEST SCORE on seed 42: 116049.10836266688


In [25]:
submission['assigned_day'] = final + 1
submission.to_csv('submission.csv')